In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy.stats import zscore

# 데이터 부ㄹ러오기
data = pd.read_csv("../data/raw/ai4i2020.csv")

In [2]:
# 이상치 처리 (Z-score 기준)
numeric_cols = [
    "Air temperature [K]", 
    "Process temperature [K]", 
    "Rotational speed [rpm]", 
    "Torque [Nm]", 
    "Tool wear [min]"
]

# z-score 절대값이 3 이상인 행 제거
data = data[(np.abs(zscore(data[numeric_cols])) < 3).all(axis=1)]

In [3]:
# 범주형 변수 원-핫 인코딩
categorical_cols = ["Type"]
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [4]:
# 타겟 / 피처 분리
X = data.drop(columns=[
    "UDI", "Product ID", "Machine failure", "TWF", "HDF", "PWF", "OSF", "RNF"
])
y = data["Machine failure"]

In [5]:
# 클래스 불균형 처리

# 고장 데이터 비율이 낮기 때문에 실제 고장 데이터를 참고해 증강
fault_data = data[data["Machine failure"] == 1]

# 각 수치형 컬럼의 평균값 기반으로 새로운 샘플 생성 (단순 증강)
n_aug = 1000  # 증강할 샘플 수
augmented_samples = pd.DataFrame({
    col: np.random.normal(loc=fault_data[col].mean(), scale=fault_data[col].std(), size=n_aug)
    for col in numeric_cols
})

In [6]:
# Type 컬럼 처리
for t in [col for col in X.columns if "Type_" in col]:
    augmented_samples[t] = 0
# 랜덤으로 한 컬럼만 1로 설정
augmented_samples["Type_L"] = 1

# 타겟 라벨 1
augmented_y = pd.Series([1]*n_aug)

# 기존 데이터와 합치기
X = pd.concat([X, augmented_samples], axis=0).reset_index(drop=True)
y = pd.concat([y, augmented_y], axis=0).reset_index(drop=True)

print("증강 후 데이터 크기:", X.shape, y.shape)

증강 후 데이터 크기: (10822, 7) (10822,)


In [7]:
# 학습/검증/테스트 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [8]:
data['Machine failure'].value_counts()

Machine failure
0    9529
1     293
Name: count, dtype: int64

In [9]:
# 수치형 컬럼 정규화
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [10]:
# 데이터 확인
print("학습 데이터:", X_train.shape, y_train.shape)
print("검증 데이터:", X_val.shape, y_val.shape)
print("테스트 데이터:", X_test.shape, y_test.shape)

학습 데이터: (7575, 7) (7575,)
검증 데이터: (1623, 7) (1623,)
테스트 데이터: (1624, 7) (1624,)


In [11]:
# 데이터 저장
X_train.to_csv("../data/X_train.csv", index=False)
X_val.to_csv("../data/X_val.csv", index=False)
X_test.to_csv("../data/X_test.csv", index=False)
y_train.to_csv("../data/y_train.csv", index=False)
y_val.to_csv("../data/y_val.csv", index=False)
y_test.to_csv("../data/y_test.csv", index=False)